In [ ]:
import pandas as pd

In [ ]:
# Functions for solving time of concentration
def kirpich( length, slope):
    tc = 0.0078 * length ** 0.77 * slope**-0.385
    return max(tc, 5), 'kirpich' # Sets the min. tc to 5mins

def faa(length, slope, c):
    slope = slope * 100
    tc = (1.8 * (1.1 - c) * length**0.5) / slope**0.33
    tc = max(tc, 5) # Sets the min. tc to 5mins
    return max(tc, 5), 'faa' # Sets the min. tc to 5mins

def scs( cn, length, slope):
    slope = slope * 100
    tc = (100 * length** 0.8 * ((1000 / cn)-9)**0.7) / (1900 * slope**0.5)
    tc = max(tc, 5) # Sets the min. tc to 5mins
    return max(tc, 5), 'scs' # Sets the min. tc to 5mins

def i_izzard( a, d , b, rc, length, slope, i_iter):
    tc = (41.025 * ((0.0007 * i_iter) + rc) * length**0.33) / (slope**(1/3) * i_iter**(2/3))
    i_calc_mm = a * (tc + d)**b
    i_calc = i_calc_mm / 10 / 2.54
    return i_calc , i_iter

def izzard( a, d , b, rc, length, slope, _threshold):
    lower = 0
    upper = 5000
    solve = (lower + upper) / 2
    threshold = i_izzard(a, d , b, rc, length, slope, solve)[0] - solve  # Compute initial threshold
    
    i_calc_val = 1 # This is just a placeholder variable for one of Izzard's condition
    while abs(threshold) >= _threshold and (i_calc_val * length) < 500:        
        if threshold < 0:
            upper = solve
        elif threshold > 0:
            lower = solve
        # Update solve based on new bounds
        solve = (lower + upper) / 2
        # Recompute threshold with updated solve
        threshold = i_izzard(a, d , b, rc, length, slope, solve)[0] - solve
        i_calc_val = solve / 10 / 2.54

    tc = (41.025 * ((0.0007 * solve) + rc) * length**0.33) / (slope**(1/3) * solve**(2/3))
    return max(tc, 5), 'izzard' # Sets the min. tc to 5mins

def i_kinematic( a, d, b, length, slope, n, i_iter):
    # Perform calculations
    tc = (0.94 * (length ** 0.6 * n ** 0.6)) / ( i_iter ** 0.4 * slope ** 0.33)
    i_calc_mm = a * (tc + d) ** b
    i_calc = i_calc_mm / 10 / 2.54
    return i_calc, i_iter

def kinematic( a, d, b,  n, length, slope, _threshold):
    lower = 0
    upper = 1000
    solve = (lower + upper) / 2
    threshold = i_kinematic(a, d, b, length, slope, n, solve)[0] - solve

    threshold_plot = []
    while abs(threshold) >= _threshold:
        if threshold < 0:
            upper = solve
        elif threshold > 0:
            lower = solve
        solve = (lower + upper) / 2
        threshold = i_kinematic(a, d, b, length, slope, n, solve)[0] - solve
        threshold_plot.append(threshold)

    tc = (0.94 * (length ** 0.6 * n ** 0.6)) / (solve ** 0.4 * slope ** 0.33)
    tc = max(tc, 5) # Sets the min. tc to 5mins
    return max(tc, 5), 'kinematic' # Sets the min. tc to 5mins

def time_of_conc(a, d, b, rc, n, cn, slope, area, l, c, _threshold):
    # Determine what applicable method should be used for tc
    # Append the applicable tc to tc_list
    # Return the max tc

    tc_list = [] # Intialize a list of time of concentration
    area_acres = area * 2.47105 # converts
    
    if 3 <= slope*100 <= 10 and area_acres <= 112:
        tc_list.append(kirpich(length=l, slope=slope))
    
    if slope > 0 and area_acres < 2000:  # Condition for Izzard (1946)
        tc_list.append(izzard(a, d, b, rc, l, slope, _threshold))
    
    if slope > 0 and area_acres > 112:  # Condition for Federal Aviation Admin (1970)
        tc_list.append(faa(l, slope, c))
    
    # Always compute for the Kinematic Wave Formula
    tc_list.append(kinematic(a, d, b, n, l, slope, _threshold))
    
    if slope < 3 and area_acres <= 2000:  # Condition for SCS Lag Equation (1975)
        tc_list.append(scs(cn, l, slope))
    
    return min(tc_list, key= lambda tc: tc[0])

In [31]:
# DataFrame of runoff coefficient || df_runoff_c
dict_runoff_c = {
        'class_run_c': ['AS', 'CN', 'GPF', 'GPA', 'GPS', 'GFF', 'GFA', 'GFS', 'GGF', 'GGA', 'GGS', 'CLF', 'CLA', 'CLS',
            'PRF', 'PRA', 'PRS', 'FWF', 'FWA', 'FWS'],
        2: [0.73, 0.75, 0.32, 0.37, 0.40, 0.25, 0.33, 0.37, 0.21, 0.29, 0.34, 0.31, 0.35, 0.39, 0.25, 0.33, 0.37, 0.20, 0.31, 0.35],
        5: [0.77, 0.80, 0.34, 0.40, 0.43, 0.28, 0.36, 0.40, 0.23, 0.32, 0.37, 0.34, 0.38, 0.42, 0.28, 0.36, 0.40, 0.25, 0.34, 0.39],
        10: [0.81, 0.83, 0.37, 0.43, 0.45, 0.30, 0.38, 0.42, 0.25, 0.35, 0.40, 0.36, 0.41, 0.44, 0.30, 0.38, 0.42, 0.28, 0.36, 0.41],
        15: [0.83, 0.85, 0.38, 0.44, 0.46, 0.31, 0.39, 0.43, 0.26, 0.36, 0.41, 0.37, 0.42, 0.45, 0.31, 0.39, 0.43, 0.29, 0.37, 0.42],
        25: [0.86, 0.88, 0.40, 0.46, 0.49, 0.34, 0.42, 0.46, 0.29, 0.39, 0.44, 0.40, 0.44, 0.48, 0.34, 0.42, 0.46, 0.31, 0.40, 0.45],
        50: [0.90, 0.92, 0.44, 0.49, 0.52, 0.37, 0.45, 0.49, 0.32, 0.42, 0.47, 0.43, 0.48, 0.51, 0.37, 0.45, 0.49, 0.35, 0.43, 0.48],
        100: [0.95, 0.97, 0.47, 0.53, 0.55, 0.41, 0.49, 0.53, 0.36, 0.46, 0.51, 0.47, 0.51, 0.54, 0.41, 0.49, 0.53, 0.39, 0.47, 0.52],
        500: [1.00, 1.00, 0.58, 0.61, 0.62, 0.58, 0.58, 0.60, 0.49, 0.56, 0.58, 0.57, 0.60, 0.61, 0.53, 0.58, 0.60, 0.48, 0.56, 0.58]
    }
df_run_temp = pd.DataFrame(dict_runoff_c).add_suffix('-yr_run_c', axis=1)
df_runoff_c = df_run_temp.rename(columns={'class_run_c-yr_run_c' : 'class_run_c'})

In [ ]:
# Read for Tullahan River Files
tullahan_river_cn = r"D:\AMH Philippines, Inc\PP25.058 NLEX FLOOD MITIGATION STUDY AROUND CASILI CREEK-MAYPAJO-VITAS - Main\05 WORK FILES\04 QGIS\03 TIFF\rdtc_temp\hydro_elements\tullahan_river_cn.csv"
tullahan_river_charac = r"D:\AMH Philippines, Inc\PP25.058 NLEX FLOOD MITIGATION STUDY AROUND CASILI CREEK-MAYPAJO-VITAS - Main\05 WORK FILES\04 QGIS\03 TIFF\rdtc_temp\hydro_elements\tullahan_river_hms_charac.csv"

In [39]:
# Helper function for weighted averaging subbasin parameters get_w_ave(dataframe)
def get_w_ave_df(dataframe):
    df_h = dataframe
    # Multiply each parameters with the area
    df_h['mult-retardance-c'] = df_h['area_has'] * df_h['ret-c']
    df_h['mult-roughness-n'] = df_h['area_has'] * df_h['n_value']
    df_h['mult-cn'] = df_h['area_has'] * df_h['CN']

    df_weighted_average = pd.DataFrame()
    # Compute for the Series
    series_ret_c = df_h.groupby('subbasin')['mult-retardance-c'].agg('sum')
    series_mannings = df_h.groupby('subbasin')['mult-roughness-n'].agg('sum')
    series_cn = df_h.groupby('subbasin')['mult-cn'].agg('sum')

    # Popluate the columns
    df_weighted_average['area_has'] = df_h.groupby('subbasin').area_has.agg('sum')
   #  df_weighted_average['w_ret-c'] = series_ret_c / df_weighted_average['area_has'] 
   #  df_weighted_average['w_mannings'] = series_mannings / df_weighted_average['area_has'] 
    df_weighted_average['w_cn'] = series_cn / df_weighted_average['area_has']

    # Weighted Average for Runoff Coefficient
    # List of runoff coefficient columns
    col_list = ['2-yr_run_c', '5-yr_run_c',
       '10-yr_run_c', '15-yr_run_c', '25-yr_run_c', '50-yr_run_c',
       '100-yr_run_c', '500-yr_run_c']
    
    # for rp in col_list:
    #     # Initialize a temporary Dataframe to Filter needed items in the original Dataframe
    #     df_temp = dataframe['area_has', rp]
    #     df_temp['mult-area-rp'] = df_temp['area_has'] * df_temp[rp]


    return df_weighted_average

In [ ]:
# Helper function Updates the runoff coefficient values based on flowpath_slope || update_class_run(row)
def assign_runoff(class_name, flowpath_slope):
    slope_pct = flowpath_slope * 100
    
    # Grassland
    if class_name == 'Grassland':
        if slope_pct < 2:
            return 'GGF'
        elif slope_pct > 7:
            return 'GGS'
        else:
            return 'GGA'
        
    # Brush/Shrubs
    elif class_name == 'Brush/Shrubs':
        if slope_pct < 2:
            return 'GFF'
        elif slope_pct > 7:
            return 'GFS'
        else:
            return 'GFA'
        
    # Crops
    elif class_name in ['Annual Crop', 'Perennial Crop']:
        if slope_pct < 2:
            return 'PRF'
        elif slope_pct > 7:
            return 'PRS'
        else:
            return 'PRA'
    
    # Open Forest
    elif class_name == 'Open Forest':
        if slope_pct < 2:
            return 'FWF'
        elif slope_pct > 7:
            return 'FWS'
        else:
            return 'FWA'    
    
    # if no condition matches, return None
    return None


def update_class_run(row):
    """Return the updated class-run- value or the old one if None."""
    new_val = assign_runoff(row['class_name'], row['flowpath_slope'])
    return new_val if new_val is not None else row['class_run-']

In [46]:
# Solve for SCS time of concentration
def scs_lag(length, cn, slope):
    tc = (100 * length** 0.8 * ((1000 / cn)-9)**0.7) / (1900 * slope**0.5)
    return tc

In [ ]:
# Tullahan River
df_tullahan_cn = pd.read_csv(tullahan_river_cn)
df_tullahan_cn.rename(columns={'subbasinna' : 'subbasin'}, inplace=True)
df_tullahan_charac = pd.read_csv(tullahan_river_charac)
col_list = ['subbasin', 'flowpath_km','flowpath_slope']
df_tullahan_trunc = df_tullahan_charac[col_list]

# Merged to HMS Charac to CN 
df_tullahan_merged = pd.merge(df_tullahan_cn, df_tullahan_trunc, on=['subbasin'])
df_tullahan_merged['flowpath_km'] = df_tullahan_merged['flowpath_km'].apply(
                                    lambda row: row * 1000 )
# Update values of runoff coefficient
df_tullahan_merged['class_run-'] = df_tullahan_merged.apply(update_class_run, axis=1)
df_tullahan_merged.rename(columns={'class_run-' : 'class_run_c'}, inplace=True) # Rename the column

# Add the Runoff Coefficient Values
df_tullahan_merged = pd.merge(df_tullahan_merged,df_runoff_c, on=['class_run_c'], suffixes=('-yr_run_c','dsadsa'))

# Perform Subbasin Summary
df_tullahan_1 = get_w_ave_df(df_tullahan_merged)
df_merging = df_tullahan_merged[['subbasin','flowpath_km', 'flowpath_slope']]
df_tullahan_2 = pd.merge(df_tullahan_1, df_merging, on=['subbasin'])
df_tullahan_2['tc_scs'] = df_tullahan_2.apply(
    lambda row: scs_lag(
        row['flowpath_km'],
        row['w_cn'],
        row['flowpath_slope']
    ),
    axis=1
)